In [1]:
import clickhouse_connect

# Get client connection
client = clickhouse_connect.get_client(host='localhost', port=8123, username='default', password='default')


In [5]:
import time
print("Querying view_aggregated_data for timing...")
query_start=time.perf_counter()
result=client.query("SELECT * FROM view_aggregated_data")
query_elapsed_ms=(time.perf_counter()-query_start)*1000


Querying view_aggregated_data for timing...


In [8]:
import pandas as pd
col_names=result.column_names
data=pd.DataFrame(result.result_rows)
data.columns=col_names

In [9]:
data

,maid,total_geohashes,total_pings,geohash,pings_array,mean_lat,mean_lon,entropy_hour_norm,home_score,work_score,leisure_score,pingsink_score,std_geohash_m
0,Fuf3u3GbIedK4ohTKwJocyHcr/q+nJOkNaK4a9H13JKxH7...,1,10,[ey5177x],[10],[33.989454650878905],[-6.877678155899048],[0.5335745571801561],[0.055620045078684746],[0.24401990270812562],[0.15812523959847458],[0.52168432061001],[7.35482114557026]
1,LWxTcoC9lpCHvpJUI211Ra+c5G7Mrzgj9ODquHTDdaYXsw...,1,10,[evfqwhd],[10],[33.511820220947264],[-7.819125938415527],[-0.0],[0.021785782663859863],[0.021653017252913784],[0.1538365104612249],[0.6894713656741316],[1.6631071950081888]
2,xhYhx171W84PC+YbXLv5WQVdCLSGukg7VCFtIoqkfdilrE...,3,12,"[evfxh6v, ey7g7s4, evfwgr1]","[2, 5, 5]","[33.58997917175293, 35.75, 33.56999588012695]","[-7.540019989013672, -5.820000171661377, -7.59...","[-0.0, 0.15745432821970826, 0.2990077632398405]","[0.023361928554345948, 0.04139896188899199, 0....","[0.00950348737078084, 0.014567933701226906, 0....","[0.055840618438907794, 0.10586607436238939, 0....","[0.0, 0.0, 0.0]","[2.5837540281733324, 0.0, 4.335193209766412]"
3,0iO65biGua/W599r2o69i8YQYMXkXYuBvwiA9vFAyr+xZg...,1,1,[evd7cwp],[1],[31.629819869995117],[-8.010100364685059],[-0.0],[0.0338981276196257],[0.005397049170301682],[0.02641346301255361],[0.0],[0.0]
4,kGdSMyOMFUbsEOaC9RXZAlHYbk/cHl2NWtG0o2CfCak+EO...,1,1,[evfxh6v],[1],[33.59000015258789],[-7.539999961853027],[-0.0],[0.005208640598821776],[0.02980452320292592],[0.026446467155969305],[0.0],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199413,eo+xRMEgT9eEHuIj2dUiIa1Sf1PY2Tld8PBc+xu6kp7wLS...,4,13,"[ev3jj9z, ev3jj9w, ev3jj9c, evd7cwp]","[4, 1, 4, 4]","[30.420087337493896, 30.418899536132812, 30.41...","[-9.591212511062622, -9.592900276184082, -9.59...","[0.17694179377516528, -0.0, -0.0, -0.0]","[0.02323590469411147, 0.005208640598821776, 0....","[0.1001292791623982, 0.02980452320292592, 0.01...","[0.12680702394172108, 0.026446467155969305, 0....","[0.0, 0.0, 0.0, 0.0]","[6.915772718656329, 0.0, 5.363719540940295, 3...."
199414,OXJkZKWGbmChpSkwXVF6AFabvuxvq07asDH+JLAMIJah+Q...,1,1,[ey56cuf],[1],[34.2599983215332],[-6.599999904632568],[-0.0],[0.005208640598821776],[0.02980452320292592],[0.026446467155969305],[0.0],[0.0]
199415,R7mJl+RVsFD2EYUbvk6s0I21vZgKh/1kL17Kx+cZK1D03N...,1,1,[evfwuzq],[1],[33.570552825927734],[-7.516547679901123],[-0.0],[0.005208640598821776],[0.005397049170301682],[0.03434221415216218],[0.0],[0.0]
199416,RvNwcglWS1dELWv1cpPYmg3CLU1hyD1oGB37GSqknHxeUw...,1,53,[evd7fht],[53],[31.622445844254404],[-7.990050900657222],[0.7383584953948619],[0.16278037021144856],[0.24875005073578232],[0.16342776204359247],[0.7023599213517666],[4.971196688806667]
